In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


In [2]:
dataPath = './data/OWID_vaccinations.csv'

df = pd.read_csv(dataPath)

df.date = pd.to_datetime(df.date)
dkdf = df[df.location == 'Denmark']

In [3]:
dkdf.columns

Index(['location', 'iso_code', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'daily_vaccinations_per_million'],
      dtype='object')

In [4]:
fig,ax1 = plt.subplots()

ax1.plot(dkdf.date,dkdf.people_vaccinated_per_hundred)
ax1.plot(dkdf.date,dkdf.people_fully_vaccinated_per_hundred)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
ifrVaccDF = pd.read_csv('IFRofVaccPerRegion.csv')



In [6]:
curDates = dkdf.date.values.astype('datetime64[D]')

dkVacc = dkdf.people_vaccinated_per_hundred
dkVacc[dkVacc.isna()] = 0

dkVacc = dkVacc.astype(int)
# dkVacc = (dkVacc/100)

dkVacc = dkVacc.values

# dkVacc

<ipython-input-6-85f2937a5009>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dkVacc[dkVacc.isna()] = 0
C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [7]:
# ifrVaccDF['Denmark']
# dkVacc.isin(ifrVaccDF['VaccRate'])
# dkVacc

ifrpertime = []

for k in range(0,len(dkVacc)):
    curVacc = dkVacc[k]
    curIndex = np.where(np.round((ifrVaccDF['VaccRate']*100)).astype(int) == curVacc)[0]
    curIFR = ifrVaccDF['Denmark'].iloc[curIndex].values[0]
    ifrpertime.append(curIFR)
    # print(curIFR)

In [8]:
# (ifrVaccDF['VaccRate'] == curVacc).values
# ifrVaccDF['VaccRate'].values
# curVacc

vaccDates = curDates + np.timedelta64(14,'D')
# plt.figure()

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(curDates,dkVacc)
# ax2.plot(curDates,ifrpertime)
ax2.plot(vaccDates,ifrpertime)


ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

plt.tight_layout()

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

# ifrpertime
# np.where((ifrVaccDF['VaccRate']*100).astype(int) == 30)
# np.round((ifrVaccDF['VaccRate']*100)).astype(int).iloc[29]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataPath = './data/OWID_data.csv'

dfData = pd.read_csv(dataPath)

# print(dfData.columns)

dfData.date = pd.to_datetime(dfData.date)
curdfData = dfData[dfData.location == 'Denmark']


In [10]:
# fig,ax1 = plt.subplots()

# ax1.plot(curdfData.date,curdfData.new_deaths)
# ax1.plot(rnTime(curdfData.date,7),rnMean(curdfData.new_deaths,7))


In [11]:
dataDates = curdfData.date.values.astype('datetime64[D]')
dataDates
curDates

missingDates = np.arange(dataDates[0],curDates[0])
missingDates

allDates = np.append(missingDates,curDates)
allDkVacc = np.append(np.zeros((len(missingDates),)),dkVacc)


ifrpertime = []

for k in range(0,len(allDkVacc)):
    curVacc = allDkVacc[k]
    curIndex = np.where(np.round((ifrVaccDF['VaccRate']*100)).astype(int) == curVacc)[0]
    curIFR = ifrVaccDF['Denmark'].iloc[curIndex].values[0]
    ifrpertime.append(curIFR)
    # print(curIFR)

fig,ax1 = plt.subplots()

# ax1.plot(allDates,allDkVacc)
ax1.plot(allDates,ifrpertime)
# ax1.plot(curDates,np.zeros((len(missingDates),)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
curdfData.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [13]:
# allDates[-1]
# len(allDates)
# len(curdfData.date.values)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(14,14))

ax1.plot(curdfData.date,curdfData.new_deaths,'k*:',markersize=2,linewidth=0.5)
ax1.plot(rnTime(curdfData.date,7),rnMean(curdfData.new_deaths,7),'k')

# ax1.plot(curDates,dkVacc,'k')

deathsToUse = curdfData.new_deaths.values[:-1]
curCase = deathsToUse/(np.divide(ifrpertime,100))
curCaseMean = deathsToUse/(ifrpertime[0]/100)


DatesToPlot = allDates - np.timedelta64(21,'D')


# ax2.plot(DatesToPlot,curCaseMean,'g*:',markersize=2,linewidth=0.5)
ax2.plot(rnTime(DatesToPlot,7),rnMean(curCaseMean,7),'g:',label='Fra konstant IFR')
# ax2.plot(DatesToPlot,curCase,'b*:',markersize=2,linewidth=0.5)
ax2.plot(rnTime(DatesToPlot,7),rnMean(curCase,7),'b--',label='Fra IFR som funktion af vaccination')
# ax2.plot(curdfData.date,curdfData.new_cases,'k*:',markersize=2,linewidth=0.5)
ax2.plot(rnTime(curdfData.date,7),rnMean(curdfData.new_cases,7),'k',label='Officielle tal')


ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

# ax2.set_xlim(left=np.datetime64('2021-02-01'))
# ax2.set_xlim(right=np.datetime64('2021-06-01'))
# ax2.set_ylim(top=1500)

ax2.legend()

ax1.set_ylabel('Dødsfald')
ax2.set_ylabel('Bekræftede tilfælde')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesFromVaccIFR_DK_old')
    
ax2.set_xlim(left=np.datetime64('2021-01-01'))
ax2.set_xlim(right=np.datetime64('2021-06-01'))
ax2.set_ylim(top=2000)

if saveFigures:
    plt.savefig('figs/CasesFromVaccIFRZoom_DK_old')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# allDates[-1]
# len(allDates)
# len(curdfData.date.values)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(14,14),gridspec_kw={'height_ratios': [3, 1]})

meanWidth = 14

deathsToUse = curdfData.new_deaths.values[:-1]
curCase = deathsToUse/(np.divide(ifrpertime,100))
curCaseMean = deathsToUse/(ifrpertime[0]/100)


DatesToPlot = allDates - np.timedelta64(21,'D')


# ax1.plot(curdfData.date,curdfData.new_cases,'k*:',markersize=2,linewidth=0.5)
ax1.plot(rnTime(curdfData.date,meanWidth),rnMean(curdfData.new_cases,meanWidth),'k',label='Confirmed cases')
# ax1.plot(DatesToPlot,curCaseMean,'g*:',markersize=2,linewidth=0.5)
ax1.plot(rnTime(DatesToPlot,meanWidth),rnMean(curCaseMean,meanWidth),'g:',label='From constant IFR')
# ax1.plot(DatesToPlot,curCase,'b*:',markersize=2,linewidth=0.5)
ax1.plot(rnTime(DatesToPlot,meanWidth),rnMean(curCase,meanWidth),'b--',label='From IFR as a function of vaccination')


ax2.plot(allDates,ifrpertime,'b--')
ax2.plot(allDates,np.ones(np.array(ifrpertime).shape)*ifrpertime[0],'g:')

ax2_2 = ax2.twinx()
clrTwin = 'm'
ax2_2.plot(curDates,dkVacc,'k',color=clrTwin)
ax2_2.set_ylabel('Vaccination rate [%]',color=clrTwin)
ax2_2.tick_params(axis='y', labelcolor=clrTwin)

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax2_2.set_ylim(bottom=0,top=100)


ax1.legend(fontsize=16)

ax2.set_ylabel('Effective IFR [%]')
ax1.set_ylabel('Cases')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax1.set_title('Denmark')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesFromVaccIFR_DK')
    
ax1.set_xlim(left=np.datetime64('2021-01-01'))
ax1.set_xlim(right=np.datetime64('2021-07-01'))
ax1.set_ylim(top=2000)

if saveFigures:
    plt.savefig('figs/CasesFromVaccIFRZoom_DK')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
# allDates[-1]
# len(allDates)
# len(curdfData.date.values)
fig,ax1 = plt.subplots(1,1,sharex=True,figsize=(14,14))

# ax1.plot(curdfData.date,curdfData.new_deaths,'k*:',markersize=2,linewidth=0.5)
# ax1.plot(rnTime(curdfData.date,7),rnMean(curdfData.new_deaths,7),'k')


DatesToPlot = curdfData.date - np.timedelta64(21,'D')


# ax2.plot(curdfData.date,curdfData.new_cases,'k*:',markersize=2,linewidth=0.5)

curDeaths = rnMean(curdfData.new_deaths,7)
curCase = rnMean(curdfData.new_cases,7)
ax1.plot(rnTime(DatesToPlot,7),np.divide(curDeaths,40),'b')
ax1.plot(rnTime(curdfData.date,7),np.divide(curCase,4000),'k',label='Officielle tal')


# ax1.set_ylim(bottom=0)

# ax1.set_ylabel('Dødsfald')
# ax2.set_ylabel('Bekræftede tilfælde')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# curdf = dfData[dfData.location == 'Peru']
# curdf.columns

# plt.figure()

# plt.plot(curdf.date,curdf.weekly_hosp_admissions,'*')
# # curdf.weekly_hosp_admissions